In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil



import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
#os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import metrics as metrics
from optimizers import *
import schedulers as schedulers

import training_utils as training_utils

import enformer_performer as enformer_performer

from scipy import stats


2023-02-22 20:08:10.545163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 20:08:10.913393: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-22 20:08:10.913442: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-22 20:08:11.979542: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-5')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy=\
        tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    options.experimental_threading.max_intra_op_parallelism=1
    mixed_precision.set_global_policy('mixed_bfloat16')
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1 # batch size 24, use LR ~ 2.5 e -04
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS
    

2023-02-14 14:53:43.584486: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-14 14:53:43.584540: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-14 14:53:43.584569: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2023-02-14 14:53:43.916129: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 14:53:43.942912: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:447] Started server with target: grpc://localhost:33697
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-5


INFO:tensorflow:Initializing the TPU system: node-5


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
iterators={'human': ('gs://genformer_data/expanded_originals/no_TF/393k',2696),
           'mouse': ('gs://genformer_data/expanded_originals/no_TF/393k',987),
           'canine': ('gs://genformer_data/expanded_originals/no_TF/393k',13),
           'rhesus': ('gs://genformer_data/expanded_originals/no_TF/393k',15),
           'rat': ('gs://genformer_data/expanded_originals/no_TF/393k',13)}
           
g = tf.random.Generator.from_non_deterministic_state()
tr_data_it_dict,val_data_it_dict,val_data_TSS_it =  \
    training_utils.return_distributed_iterators(iterators,
                                                "gs://genformer_data/expanded_originals/no_TF/393k/human/tfrecords_tss",
                                                2696,
                                                 GLOBAL_BATCH_SIZE,
                                                 393216,
                                                 3072,
                                                 1088,
                                                 10,
                                                 4,
                                                 10,
                                                 strategy,
                                                 options,
                                                 g)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


393216
896
0
2696


In [4]:
with strategy.scope():
    model = enformer_performer.enformer_performer(
        num_transformer_layers=6,
        num_heads=8,
        heads_channels= {'human': 2696,
                         'mouse': 987,
                         'canine': 13,
                         'rhesus': 15,
                         'rat': 13},
        out_length=3072,
        target_length=896,
        stable_variant=True,
        dim=192,
        d_model=1536,
        norm=True,
        max_seq_length=3072,
        nb_random_features=256,
        hidden_size=1536,
        numerical_stabilizer=0.001,
        rel_pos_bins=3072,
        use_mask_pos=False,
        use_rot_emb=True,
        kernel_transformation="softmax_kernel_transformation",
        normalize=True,
        seed=5)

In [5]:
with strategy.scope():        

    optimizer1 = tf.keras.optimizers.Adam(learning_rate=1.0e-04)

    optimizer2 = tf.keras.optimizers.Adam(learning_rate=1.0e-04)
    optimizers_in = optimizer1,optimizer2

    metric_dict = {}
    organism_dict = {'human': 50,
                     'mouse': 50,
                     'rhesus':30,
                     'rat': 30,
                     'canine': 30}

                               
    dist_train_step, dist_val_step_h,dist_val_step_m, val_step_TSS, build_step,metric_dict = training_utils.return_train_val_functions(model,
                                                                                                                                       50,
                                                                                                                                       organism_dict,
                                                                                                                                         50,
                                                                                                                                       optimizers_in,
                                                                                                                                        2058,
                                                                                                                                         strategy,
                                                                                                                                         metric_dict, 
                                                                                                                                         GLOBAL_BATCH_SIZE,
                                                                                                                                        5.0,
                                                                                                                                         BATCH_SIZE_PER_REPLICA,
                                                                                                                                        loss_fn_main='poisson')
    


In [ ]:
with strategy.scope():        
    ### main training loop
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 4):
        print('starting epoch_', str(epoch_i))
        start = time.time()
        if epoch_i == 1:
            # run once to build the model w/o updating anything
            build_step(val_data_it_dict['human'])

        assert len(organism_dict.keys()) == len(tr_data_it_dict.keys())
            
        iters = (tr_data_it_dict['human'],
                        tr_data_it_dict['mouse'],
                        tr_data_it_dict['rhesus'],
                        tr_data_it_dict['rat'],
                        tr_data_it_dict['canine'])
        dist_train_step(iters)
        


        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('hg_train_loss: ' + str(metric_dict['human_tr'].result().numpy()))
        
        print('training duration(mins): ' + str(duration))

        start = time.time()
        dist_val_step_h(val_data_it_dict['human'])

        print('val_loss: ' + str(metric_dict['human_val'].result().numpy()))
        val_losses.append(metric_dict['human_val'].result().numpy())

        print('human_pearsonsR: ')
        pearsonsR=metric_dict['human_pearsonsR'].result()['PearsonR'].numpy()
        print(pearsonsR)

        val_pearsons.append(np.nanmedian(pearsonsR))
        print('human_R2: ')
        print(metric_dict['human_R2'].result()['R2'].numpy())


        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i) + ' validation')
        print('validation duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))

        
        val_step_TSS(val_data_TSS_it)
        

        y_trues = metric_dict['hg_corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['hg_corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['hg_corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['hg_corr_stats'].result()['gene_map'].numpy()
        
        
        
        if (epoch_i > 2):
            stop_criteria,patience_counter,best_epoch = \
                training_utils.early_stopping(current_val_loss=val_losses[-1],
                                                logged_val_losses=val_losses,
                                                current_pearsons=val_pearsons[-1],
                                                logged_pearsons=val_pearsons,
                                                current_epoch=epoch_i,
                                                best_epoch=best_epoch,
                                                save_freq=5,
                                                patience=5,
                                                patience_counter=patience_counter,
                                                min_delta=1.0e-05,
                                                model=enformer_model,
                                                save_directory="gs://picard-testing-176520/test",
                                                saved_model_basename="test_model",
                                                checkpoint=checkpoint)
        #plt.close('all')
        print('patience counter at: ' + str(patience_counter))
        for key, item in metric_dict.items():
            item.reset_state()

starting epoch_ 1


/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


In [15]:

def deserialize_val_TSS(serialized_example,
                        input_length, output_length,crop_size,
                        max_shift,num_targets):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string),
        'tss_mask': tf.io.FixedLenFeature([], tf.string),
        'gene_name': tf.io.FixedLenFeature([], tf.string),
        'bin_unique': tf.io.FixedLenFeature([], tf.string)
    }
    
    shift = 5
    input_seq_length = input_length + max_shift
    interval_end = input_length + shift
    

    example = tf.io.parse_example(serialized_example, feature_map)
    sequence = tf.io.decode_raw(example['sequence'], tf.bool)
    sequence = tf.reshape(sequence, (input_length + max_shift, 4))
    sequence = tf.cast(sequence, tf.float32)
    sequence = tf.slice(sequence, [shift,0],[input_length,-1])
    
    target = tf.io.decode_raw(example['target'], tf.float16)
    target = tf.reshape(target,
                        (output_length, num_targets))
    #print(target.shape)
    target = tf.slice(target,[crop_size,0],
                             [output_length - 2*crop_size,-1])

    bin_unique = tf.io.parse_tensor(example['bin_unique'],
                                  out_type=tf.int32)
    
    tss_mask = tf.io.parse_tensor(example['tss_mask'],
                                  out_type=tf.int32)
    tss_mask = tf.slice(tss_mask,[crop_size,0],
                             [output_length - 2*crop_size,-1])

    gene_name= tf.io.parse_tensor(example['gene_name'],out_type=tf.int32)
    gene_name = tf.tile(tf.expand_dims(gene_name,axis=0),[638])
    cell_types = tf.range(0,638)
    

    return {'sequence': tf.ensure_shape(sequence,
                                        [input_length,4]),
            'target': tf.ensure_shape(target,
                                      [output_length - 2*crop_size,num_targets]),
            'tss_mask': tf.ensure_shape(tss_mask,
                                        [output_length - 2*crop_size,1]),
            'gene_name': tf.ensure_shape(gene_name,
                                         [638,]),
            'bin_unique': bin_unique,
            'cell_types': tf.ensure_shape(cell_types,
                                           [638,])}



dataset = tf.data.TFRecordDataset("gs://genformer_data/expanded_originals_tss_mask/196k/human/tfrecords_tss/tssmask-train-0-0.tfr",
                                  compression_type='ZLIB',
                                  num_parallel_reads=4)
#dataset = dataset.with_options(options)

dataset = dataset.map(lambda record: deserialize_val_TSS(record,
                                                         196608, 
                                                        896,
                                                        0,
                                                         10,
                                                         5313),
                      deterministic=False,
                      num_parallel_calls=4)




In [16]:
test_iter = iter(dataset)


In [19]:
test = next(test_iter)

In [20]:
test

{'sequence': <tf.Tensor: shape=(196608, 4), dtype=float32, numpy=
 array([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]], dtype=float32)>,
 'target': <tf.Tensor: shape=(896, 5313), dtype=float16, numpy=
 array([[0.0402 , 0.06155, 0.     , ..., 0.     , 0.     , 0.481  ],
        [0.1345 , 0.1501 , 0.     , ..., 0.     , 0.9844 , 0.     ],
        [0.1726 , 0.3892 , 0.0251 , ..., 0.     , 0.     , 0.     ],
        ...,
        [0.0977 , 0.0776 , 0.04218, ..., 0.     , 0.483  , 0.     ],
        [0.0856 , 0.04834, 0.01191, ..., 0.     , 0.     , 0.     ],
        [0.06094, 0.08276, 0.11487, ..., 0.     , 0.     , 0.     ]],
       dtype=float16)>,
 'tss_mask': <tf.Tensor: shape=(896, 1), dtype=int32, numpy=
 array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
 